# Trend Predicting

* **Problem**: Trend Predicting based on user behaviors
* **Descript
ion**: 
  * Create online preference index for each major product group
  * Predict demand trends and draw insights by product group
  * Suggest new service from 1), 2)
* **Dataset**: ? non-numeric column, and ? numeric columns
  * ? : ?
  * Non-numeric columns:
    * 16 non-numeric columns : ?
  * Numeric columns: 
    * 104 numeric columns : ?
 


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import os

# Load Data

* 'custom.csv'
* 'search1.csv'
* 'search2.csv'
* 'master.csv'
* 'session.csv'
* 'product.csv'


In [3]:
path = Path().resolve().parents[0]
product = pd.read_csv(os.path.join(path, 'Dataset', 'Product.csv'))
search1 = pd.read_csv(os.path.join(path, 'Dataset', 'Search1.csv'))
search2 = pd.read_csv(os.path.join(path, 'Dataset', 'Search2.csv'))
custom = pd.read_csv(os.path.join(path, 'Dataset', 'Custom.csv'))
session = pd.read_csv(os.path.join(path, 'Dataset', 'Session.csv'))
master = pd.read_csv(os.path.join(path, 'Dataset', 'Master.csv'))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print('product shape : ', product.shape)
print('search1 shape : ', search1.shape)
print('search2 shape : ', search2.shape)
print('custom shape : ', custom.shape)
print('session shape : ', session.shape)
print('master shape : ', master.shape)

product shape :  (5024906, 8)
search1 shape :  (2884943, 4)
search2 shape :  (8051172, 3)
custom shape :  (671679, 3)
session shape :  (2712907, 9)
master shape :  (847652, 5)


In [5]:
product.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],"39,000",1
1,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
2,4142395,6158159,85,179538,(not set),[아베다],"39,000",1
3,4144914,7935714,12,554336,색상:블랙|사이즈:160cm(12~13세) / 1개,아디다스 키즈,"39,000",1
4,4144917,6406509,78,190306,5개,데코르테,"39,000",5


In [6]:
search1.head()

,CLNT_ID,SESS_ID,KWD_NM,SEARCH_CNT
0,5607714,7112876,빌리프 아이크림,6
1,5607714,4090791,프리메라 마스크팩,3
2,5607714,4090791,여성청결제,1
3,5612428,1876482,명품가방,1
4,5612428,658123,콩순이 아이스크림,1


In [7]:
search2.head()

,SESS_DT,KWD_NM,SEARCH_CNT
0,20180407,닥스원피스,8
1,20180407,닥터지 브라이트닝,1
2,20180407,달팡 인트랄,2
3,20180407,대상트,1
4,20180407,더블유닷,6


In [8]:
custom.head()

,CLNT_ID,CLNT_GENDER,CLNT_AGE
0,4830726,F,30
1,4830874,F,40
2,4830975,F,30
3,4831275,F,30
4,4825325,F,30


In [9]:
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,"1,890",mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,"1,604",mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,"1,035",mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298,mobile,Seoul,Seoul


In [10]:
master.head()

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
1,62282,여자 플라워덧신 2족선물세트 17403-2set,속옷/양말/홈웨어,여성양말류,여성일반양말
2,61729,88A2933253배트맨스웨트티 - 블랙 / 130,유아동의류,유아의류상의,영유아티셔츠/탑
3,61537,"닥터마틴 아드리안 블랙, 체리레드 - 02_체리레드 / 250mm(6)",패션잡화,남성화,남성부츠
4,58820,여성 그레이 스트라이프 퍼프 소매 블라우스 (128865YQ33) - 회색(앤틱실버...,남성의류,남성의류상의,남성남방셔츠


## Explore Data (1)
* Check Numeric, Categorical Columns
* One-hot encode categorical columns

In [11]:
def explore_data_1(data) :
    print(data.dtypes.value_counts())
    numeric = data._get_numeric_data().columns
    categorical = data.select_dtypes(exclude=["number"]).columns
    print('* Numeric columns : ')
    print(numeric)
    print('* Categorical columns : ')
    print(categorical)
    print('* Num of categories in categorial colums : ')
    print(data.select_dtypes('object').apply(pd.Series.nunique, axis=0))

print('[product]')
explore_data_1(product)

print('\n[search1]')
explore_data_1(search1)

print('\n[search2]')
explore_data_1(search2)

print('\n[custom]')
explore_data_1(custom)

print('\n[session]')
explore_data_1(session)

print('\n[master]')
explore_data_1(master)

#df = pd.get_dummies(df)
#print('Training Features Shape :', df.shape)

[product]
object    4
int64     4
dtype: int64
* Numeric columns : 
Index(['CLNT_ID', 'SESS_ID', 'HITS_SEQ', 'PD_C'], dtype='object')
* Categorical columns : 
Index(['PD_ADD_NM', 'PD_BRA_NM', 'PD_BUY_AM', 'PD_BUY_CT'], dtype='object')
* Num of categories in categorial colums : 
PD_ADD_NM    759548
PD_BRA_NM     22716
PD_BUY_AM     17731
PD_BUY_CT       361
dtype: int64

[search1]
int64     3
object    1
dtype: int64
* Numeric columns : 
Index(['CLNT_ID', 'SESS_ID', 'SEARCH_CNT'], dtype='object')
* Categorical columns : 
Index(['KWD_NM'], dtype='object')
* Num of categories in categorial colums : 
KWD_NM    81539
dtype: int64

[search2]
object    2
int64     1
dtype: int64
* Numeric columns : 
Index(['SESS_DT'], dtype='object')
* Categorical columns : 
Index(['KWD_NM', 'SEARCH_CNT'], dtype='object')
* Num of categories in categorial colums : 
KWD_NM        84607
SEARCH_CNT     1436
dtype: int64

[custom]
int64     2
object    1
dtype: int64
* Numeric columns : 
Index(['CLNT_ID', 'CLNT_A

* Check Data Imbalance

In [12]:
'''print(df['TARGET'].value_counts())
df['TARGET'].astype(int).plot.hist()
'''

"print(df['TARGET'].value_counts())\ndf['TARGET'].astype(int).plot.hist()\n"

# Merge Data
1) Merge product purchase data to see features which is expected to affect on purchase of client bought given product group, not individual product

- Merge following csv files
    * Product.csv : CLNT_ID, SESS_ID(Brdige), PD_C(Bridge), PD_BUY_CT, PD_BUY_AM
    * Master.csv : PD_C(Bridge), CLAC1_NM, CLAC2_NM, CLAC3_NM
    * Session.csv : SESS_ID(Bridge), SESS_DT, DVC_CTG_NM, ZON_NM, CITY_NM
    * Custom.csv : CLNT_ID, CLNT_GENDER, CLNT_AGE
    
    
- To get data below
  
  
- Client Information
    * CLNT_ID : Client ID (Primary Key)
    * CLNT_GENDER : Client Gender
    * CLNT_AGE : Client Age
    * DVC_CTG_NM : Client Device
    * ZON_NM : Client Location (briefly)
    * CITY_NM : Client Location (in detail)
    
- Purchase Information
    * SESS_DT : Session Date
    * PD_BUY_CT : Number of product purchased
    * PD_BUY_AM : Price of product purchased
    
- Product Information
    * CLAC1_NM : Product group (largest)
    * CLAC2_NM : Product group (moderate)
    * CLAC3_NM : Product group (smallest)
    
    
    
    
  

In [13]:
merge1 = pd.merge(product, master)
merge1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],"39,000",1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
1,4350334,2023661,77,642112,색상:엔젤 / 1개,바비브라운,"39,000",1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
2,5172356,4714565,47,642112,색상:카바나,[바비브라운],"39,000",1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
3,5444959,6843465,24,642112,색상:워터멜론 / 1개,바비브라운,"39,000",1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
4,5461276,7896792,53,642112,색상:워터멜론 / 1개,바비브라운,"39,000",1,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터


In [14]:
merge1 = merge1.drop(['HITS_SEQ','PD_C', 'PD_ADD_NM', 'PD_BRA_NM', 'PD_NM'], axis=1)
merge1.head()

,CLNT_ID,SESS_ID,PD_BUY_AM,PD_BUY_CT,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,4139680,7605037,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
1,4350334,2023661,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
2,5172356,4714565,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
3,5444959,6843465,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
4,5461276,7896792,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터


In [15]:
merge2 = pd.merge(merge1, session)
merge2.head()

,CLNT_ID,SESS_ID,PD_BUY_AM,PD_BUY_CT,CLAC1_NM,CLAC2_NM,CLAC3_NM,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,4139680,7605037,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,1,20180526,21.0,902,desktop,Seoul,Seoul
1,4139680,7605037,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,1,20180526,21.0,902,desktop,Seoul,Seoul
2,4139680,7605037,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,1,20180526,21.0,902,desktop,Seoul,Seoul
3,4139680,7605037,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,1,20180526,21.0,902,desktop,Seoul,Seoul
4,4139680,7605037,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,1,20180526,21.0,902,desktop,Seoul,Seoul


In [16]:
merge2 = merge2.drop(['SESS_ID', 'SESS_SEQ','TOT_PAG_VIEW_CT', 'TOT_SESS_HR_V'], axis=1)
merge2.head()

,CLNT_ID,PD_BUY_AM,PD_BUY_CT,CLAC1_NM,CLAC2_NM,CLAC3_NM,SESS_DT,DVC_CTG_NM,ZON_NM,CITY_NM
0,4139680,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,20180526,desktop,Seoul,Seoul
1,4139680,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,20180526,desktop,Seoul,Seoul
2,4139680,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,20180526,desktop,Seoul,Seoul
3,4139680,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,20180526,desktop,Seoul,Seoul
4,4139680,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,20180526,desktop,Seoul,Seoul


In [17]:
merge3 = pd.merge(merge2, custom)
merge3.head()

,CLNT_ID,PD_BUY_AM,PD_BUY_CT,CLAC1_NM,CLAC2_NM,CLAC3_NM,SESS_DT,DVC_CTG_NM,ZON_NM,CITY_NM,CLNT_GENDER,CLNT_AGE
0,4350334,"39,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,20180827,mobile,Seoul,Seoul,F,30
1,4350334,"80,000",1,화장품/뷰티케어,메이크업,BB/파운데이션/컴팩트류,20180827,mobile,Seoul,Seoul,F,30
2,4350334,"127,000",1,패션잡화,여성화,여성쪼리,20180827,mobile,Seoul,Seoul,F,30
3,4350334,"40,000",1,화장품/뷰티케어,메이크업,BB/파운데이션/컴팩트류,20180614,mobile,Seoul,Seoul,F,30
4,4350334,"40,000",1,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터,20180612,mobile,Seoul,Seoul,F,30


## Explore Data (2)
* Brief visualization

In [18]:
purchase = merge3
purchase['CLNT_ID'].value_counts()

4736937    3154
2075522    1827
1662720    1587
4964593    1262
748826     1210
5302905    1150
5142305    1030
4006944     956
5448721     828
6366359     782
6072971     722
4931190     702
5628642     678
1616624     677
3137116     661
3447474     644
2426355     643
2436343     617
3858367     616
4123774     523
3095344     507
4260668     459
1141526     447
97459       421
3803223     400
235570      381
5091280     378
3846487     353
1012545     351
2771792     346
           ... 
1233150       1
4921820       1
4356417       1
4033902       1
6240788       1
4993916       1
6251612       1
5117286       1
3052643       1
2933549       1
3719764       1
1386405       1
5151962       1
2439159       1
4646747       1
1186240       1
1153456       1
4141108       1
20740         1
6089017       1
4087111       1
3399128       1
6276130       1
3904593       1
1654981       1
964825        1
4079642       1
2408326       1
1367850       1
3760737       1
Name: CLNT_ID, Length: 6

In [20]:
purchase[purchase['CLNT_ID']==4736937]

,CLNT_ID,PD_BUY_AM,PD_BUY_CT,CLAC1_NM,CLAC2_NM,CLAC3_NM,SESS_DT,DVC_CTG_NM,ZON_NM,CITY_NM,CLNT_GENDER,CLNT_AGE
2882736,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882737,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882738,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882739,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882740,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882741,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882742,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882743,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882744,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
2882745,4736937,"52,000",1,남성의류,남성의류상의,남성티셔츠,20180426,mobile,Gyeongsangnam-do,Jinju-si,F,40
